# 3. Функции и работа с данными

In [3]:
import pandas as pd

### Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class.

In [ ]:
rating = pd.read_csv('Data_for_homeworks/HW_2_3/ratings.csv')
mov = pd.read_csv('Data_for_homeworks/HW_2_3/movies.csv')
mov_rat = mov.merge(rating, on='movieId', how='left')

In [ ]:
def classify(row):
    if row.rating >= 4.5:
      return 'высокий рейтинг'

    elif 2 < row.rating < 4.5:
      return 'средний рейтинг'

    elif row.rating <= 2:
      return 'низкий рейтинг'

In [ ]:
mov_rat['class'] = mov_rat.apply(classify, axis = 1)
mov_rat.loc[ :10]

### Задание 2
Используем файл <i>keywords.csv</i>.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

Результат классификации запишите в отдельный столбец region.

In [1]:
geo_data = {
  
  'Центр': ['москва', 'тула', 'ярославль'],
  'Северо-Запад': ['петербург', 'псков', 'мурманск'],
  'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

In [4]:
df_keys = pd.read_csv('Data_for_homeworks/HW_2_3/keywords.csv')

In [7]:
def key_region(row):
  for region, city_list in geo_data.items():
    for city in city_list:
      if city in row:
        return region

  return 'undefined'

In [6]:
df_keys['region'] = df_keys['keyword'].apply(key_region)
df_keys[df_keys['region'] != 'undefined']

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
...,...,...,...
99590,авито ярославль автомобили с пробегом,3673,Центр
99634,северпост новости мурманской области,3606,Северо-Запад
99808,полармед мурманск запись на прием,3719,Северо-Запад
99890,яндекс метро москва,4548,Центр


### Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы grouplens.org.../movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

- для каждой строки пройдите по всем годам списка years
- если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
- если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [ ]:
import matplotlib.pyplot as plt

In [5]:
# ratings = pd.read_csv('Data_for_homeworks/HW_2_2/ratings.csv')
movies = pd.read_csv('Data_for_homeworks/HW_2_3/movies.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary
9123,164977,The Gay Desperado (1936),Comedy


In [ ]:
"""
функция production_year выставляет год выпуска фильму по условию:
  - год от 1950 по 2010
  - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
  - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
"""

def production_year(film):
    years = range(1950, 2010)
    val = [x for x in years if str(x) in film['title']]
    if val:
        return val[0]
    else:
        return 1900

In [ ]:
#добавление столбца 'year'
movies['year'] = movies.apply(production_year, axis=1)
#объединение DFs и подсчёт среднего рейтинга
merged = movies.merge(ratings, on = 'movieId', how = 'left')
grouped_dfs = merged.groupby('year').rating.mean().reset_index()
grouped_dfs.sort_values('rating', ascending = False)
#исключение фильмов без года (не в range(1950, 2010)), им всем присвоен 1900-й год
grouped_dfs_excluded = grouped_dfs[grouped_dfs['year'] != 1900] 
plt.plot(grouped_dfs_excluded['year'], grouped_dfs_excluded['rating'])